In [1]:
def is_part_of_circle(x, y, circle_upper, circle_lower):
    try:
        return y > circle_upper[x] or y < circle_lower[x]
    except KeyError:
        return False
    

In [54]:
def jaccard_score(img_name):
    intersection = 0.0
    union = 0.0
    
    ground_truth = Image.open('../../ISIC-2017_Training_Part1_GroundTruth/ISIC_' + img_name + '_segmentation.png')
    rgb_ground_truth = ground_truth.convert('RGB')
    
    output_im = Image.open("./grouped_clusters_noblock/" + img_name + ".png")
    rgb_output_im = output_im.convert('RGB')
    
    output_width, output_height = output_im.size
    truth_width, truth_height = ground_truth.size
    
    width_scale_const = float(truth_width)/float(output_width)
    height_scale_const = float(truth_height)/float(output_height)
    
    for y in range(output_height):
        for x in range(output_width):
            ground_pixel = rgb_ground_truth.getpixel((x*width_scale_const, y*height_scale_const))
            output_pixel = rgb_output_im.getpixel((x, y))
            
            isGround = ground_pixel == (255,255,255)
            isOutput = output_pixel == (255,255,255)
            
            if isGround and isOutput:
                intersection += 1.0
            if isGround or isOutput:
                union += 1.0
    
    print img_name + " - " + str(intersection/union)
    

In [55]:
def print_removed_circle(img_name,circle_pixels):
    original_im = Image.open('../../ISIC-2017_Training_Data/ISIC_' + img_name + '.jpg')
    output = original_im.copy() #Image.new( 'RGB', (width,height), "black") # create a new black image
    output_pixels = output.load() # create the pixel map
    print circle_pixels
    width, height = original_im.size
    for y in range(height):
        for x in range(width):
            if x in circle_pixels and y in circle_pixels[x]:
                output_pixels[x,y] = (255, 255, 255)
    output.save("./circle_tests/" + img_name + ".png")

In [56]:
from PIL import Image
import math

def remove_circle(img_name):

    original_im = Image.open('../../ISIC-2017_Training_Data/ISIC_' + img_name + '.jpg')
    rgb_original_im = original_im.convert('RGB')
    
    width, height = original_im.size
    
    radius = int(max(width, height) / math.sqrt(2)) #550
    
    percentage_black = 1
    center_x = width/2
    center_y = height/2
    circle_pixels = {}
    
    while radius > 0 and percentage_black > 0.65:
        output = original_im.copy() #Image.new( 'RGB', (width,height), "black") # create a new black image
        output_pixels = output.load() # create the pixel map
        
        circle_upper = {}
        circle_lower = {}
        circle_pixels = {}
        
        for x in range(-radius,radius):
            y_upper = center_y + int(math.sqrt(radius*radius - x*x))
            y_lower = center_y - int(math.sqrt(radius*radius - x*x))
            x_val = center_x + x

            if x_val > 0 and x_val < width and y_upper > 0 and y_upper < height and y_lower > 0 and y_lower < height:               
                circle_upper[x_val] = y_upper
                circle_lower[x_val] = y_lower
        outside_pixels = []
        dark_ctr = 0.0
        for y in range(height):
            for x in range(width):
                try:
                    if x < (center_x-radius) or x > (center_x+radius-1) or y > circle_upper[x] or y < circle_lower[x]:
                        output_pixels[x,y] = (255, 255, 255)
                        r, g, b = rgb_original_im.getpixel((x,y))
                        avg = long((r + b + g))/3.0
                        outside_pixels.append(avg)
                        pixel_id = str(x) + "-" + str(y)
                        circle_pixels[pixel_id] = (x,y)
                            
                        if avg < 30:
                            dark_ctr += 1.0
                except KeyError:
                    continue
        var = len(outside_pixels)
        if not(var == 0):            
            percentage_black = dark_ctr/long(len(outside_pixels))
            #print "Number of pixels: " + str(len(outside_pixels)) +  " Dark counter: " + str(dark_ctr)
        radius = radius - 10
    output.save("./circle_tests/" + img_name + str(radius) + ".png")
    return circle_pixels

In [57]:
from skimage.transform import rescale
from skimage.io import imread, imshow
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

%matplotlib inline

def group_clusters(img_name, isBlocked):
    print "###########"
    print "Grouping clusters for " + img_name
    #cluster_im = Image.open()
    path = ''
    if isBlocked:
        path = '../../cmeans_block/ISIC_' + img_name + '_visual.gif'
    else:
        path = '../../cmeans_noblock/ISIC_' + img_name + '_visual.gif'
    cluster_im = Image.open(path)
    original_im = Image.open('../../ISIC-2017_Training_Data/ISIC_' + img_name + '.jpg')
    rgb_cluster_im = cluster_im.convert('RGB')
    rgb_original_im = original_im.convert('RGB')

    scale_factor = float(cluster_im.size[0]) / float(original_im.size[0])
    circle_pixels = remove_circle(img_name)
#     print_removed_circle(img_name,circle_pixels)
    print "Removed circle"
    
    forbidden_clusters = set()
    clusters = {}

    #collect pixel values for each cluster
    width, height = cluster_im.size
    for y in range(height):
        for x in range(width):
            r, g, b = rgb_cluster_im.getpixel((x, y))
            cluster_id = str(r) + '-' + str(g) + '-' + str(b)
            pixel_id = str(x) + "-" + str(y)
            
            if pixel_id in circle_pixels:
                forbidden_clusters.add(cluster_id)
            
            if cluster_id not in forbidden_clusters:
                original_rgb = rgb_original_im.getpixel((x/scale_factor, y/scale_factor)) #.3258 to deal with scale difference

                if cluster_id in clusters:
                    clusters[cluster_id].append(original_rgb)
                else:
                    clusters[cluster_id] = [original_rgb]
    
    print "# of clusters removed = " + str(len(forbidden_clusters))
    print "# of clusters = " + str(len(clusters))
    
    cluster_averages = {}

    #get average colour of clusters
    for cluster_id in clusters:
        if cluster_id not in forbidden_clusters:
            r, g, b = 0, 0, 0
            count = 0
            for pixel in clusters[cluster_id]:
                r += pixel[0]
                g += pixel[1]
                b += pixel[2]
                count += 1
            cluster_averages[cluster_id] = ((r/count), (g/count), (b/count))

    #Create Data for clustering
    list_of_pixels = []
    list_of_ids = {}
    counter = 0
    for cluster_id in cluster_averages:
        list_of_ids[cluster_id] = counter
        counter += 1
        list_of_pixels.append(list(cluster_averages[cluster_id]))
    
    print "# of averages = " + str(len(cluster_averages))
    print "# of Clusters = " + str(len(list_of_pixels))
    cvecs = np.array(list_of_pixels)
    
    print 'Clustering...'
    
    #Cluster using k-means
    kmns = KMeans(n_clusters=2, random_state=1729)
    kmns.fit(cvecs)
    
    #Find darker of centroids
    cluster1 = kmns.cluster_centers_[0]
    cluster2 = kmns.cluster_centers_[1]
    
    avg1 = (cluster1[0] + cluster1[1] + cluster1[2])/3
    avg2 = (cluster2[0] + cluster2[1] + cluster2[2])/3

    #Generate output using clustered labels

    output = Image.new( 'RGB', (width,height), "black") # create a new black image
    output_pixels = output.load() # create the pixel map
    print "Creating output image"

    width, height = cluster_im.size
    for y in range(height):
        for x in range(width):
            r, g, b = rgb_cluster_im.getpixel((x, y))
            cluster_id = str(r) + '-' + str(g) + '-' + str(b)
            if cluster_id in list_of_ids:
                index = list_of_ids[cluster_id]
                if kmns.labels_[index] and avg2 < avg1:
                    output_pixels[x,y] = (255, 255, 255) # set the colour accordingly
                elif not kmns.labels_[index] and avg1 < avg2:
                    output_pixels[x,y] = (255, 255, 255)
    
    print "Process Complete"
    plt.imshow(output)
    output.save("./grouped_clusters/" + img_name + ".png")

In [60]:

isBlocked = True

#group_clusters('0000006')
# group_clusters('0000011')
# group_clusters('0000030')
# #group_clusters('0000036')
# group_clusters('0000045')
# group_clusters('0000169')
# group_clusters('0000047')
# group_clusters('0000125')
# group_clusters('0009564')
# group_clusters('0014582')


#for photo_number in range(0, 15295):    
for photo_number in range(0, 45):    
    try:
        id = str(photo_number)
        new_id = ''

        num_digits = len(id)

        if num_digits < 7:
            difference = 7 - num_digits

            for digit in range(0, difference):
                new_id = new_id + '0'
            id = new_id + id

        #result = group_clusters(id)
        jaccard_score(id)
        
    except IOError:
        continue


# jaccard_score('0000000')
# jaccard_score('0000011')
# jaccard_score('0000030')



0000000 - 0.983149079582
0000001 - 0.893114985033
0000002 - 0.527084806249
0000003 - 0.962500901421
0000004 - 0.00443222999097
0000006 - 0.384171989406
0000007 - 0.505465164789
0000008 - 0.926514005905
0000009 - 0.843535522167
0000010 - 0.726758887401
0000011 - 0.102248875562
0000012 - 0.642046547413
0000013 - 0.918265709682
0000014 - 0.794462892887
0000015 - 0.767518009168
0000016 - 0.969549970777
0000017 - 0.955663862904
0000018 - 0.936324869161
0000019 - 0.857116181798
0000020 - 0.466721194152
0000021 - 0.921188747731
0000022 - 0.631308175767
0000023 - 0.928291092929
0000024 - 0.334295924629
0000025 - 0.756617019258
0000026 - 0.636962207991
0000027 - 0.936504927976
0000028 - 0.918590460694
0000029 - 0.956980860383
0000030 - 0.366346988713
0000031 - 0.206336506921
0000032 - 0.0
0000034 - 0.853032576082
0000035 - 0.778755158014
0000036 - 0.167667519853
0000037 - 0.556808398367
0000038 - 0.812818022019
0000039 - 0.971735909116
0000040 - 0.439699774102
0000041 - 0.894801980198
0000042 -